In [6]:
import os
import torch
import torch.nn as nn 
from utils.common import setup_seed 
from utils.loader import init_dataset_train, init_dataset_test
from utils.metrics import metrics
from MLPN.model import CSWinTransv2_threeIn
from MLPN.utils import extract_feature, get_id, extract_feature

class MLPN_:
    def __init__(self) -> None:
        # default parameters
        self.seed = 2024
        self.pytorch_version =  torch.__version__
        
        self.LPN = True
        self.lr = 0.005
        self.num_epochs = 210
        self.batchsize = 8
        self.warm_epoch = 0
        self.extra_Google = False
        self.block = 4
        self.checkpoint_interval = 10
        self.w = 256
        self.h = 256
        self.droprate = 0.75

        self.decouple = False
        self.infonce = 1

        # init
        setup_seed(self.seed)
        self.model_dir = os.path.join(os.getcwd(), 'model', 'MLPN')
        os.environ['TORCH_HOME']='./'        

    def train(self, style='mixed'):
        image_datasets, dataloaders, dataset_sizes = init_dataset_train(batchsize=self.batchsize, style=style)


    def test(self, pth=None, query='drone', gallery='satellite', multiple_scale=[1], batchsize=128, style='mixed'):
        # load data
        image_datasets, dataloaders, dataset_sizes = init_dataset_test(batchsize=batchsize, style=style, w=self.w, h=self.h)
        # init label
        gallery_name = 'gallery_' + gallery
        query_name = 'query_' + query 
        gallery_label = get_id(image_datasets[gallery_name].imgs)
        query_label = get_id(image_datasets[query_name].imgs)
        # print(dataset_sizes[gallery_name])
        # load model
        model_file = os.listdir(self.model_dir)[-1] if pth==None else pth + '.pth'
        print("load model: {}".format(model_file))
        model = CSWinTransv2_threeIn(701, droprate=self.droprate, decouple=self.decouple, infonce=self.infonce)
        

        try:
            model.load_state_dict(torch.load(os.path.join(self.model_dir, model_file))) 
        except:
            model = nn.DataParallel(model)
            model.load_state_dict(torch.load(os.path.join(self.model_dir, model_file))) 
            model = model.module

        if self.LPN:
            for i in range(self.block):
                cls_name = 'classifier'+str(i)
                c = getattr(model, cls_name)
                c.classifier = nn.Sequential()
        else:
            model.model.classifier.classifier = nn.Sequential()

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        model = model.to(device)
        # model.model = model.model.to(device)
        model = model.eval()

        # extract features
        with torch.no_grad():
            query_feature = extract_feature(model,dataloaders[query_name], view=query, ms=multiple_scale)
            gallery_feature = extract_feature(model,dataloaders[gallery_name], view=gallery, ms=multiple_scale)

        # calculate 
        m = metrics(query_feature, query_label, gallery_feature, gallery_label)
        print("Recall@1: {:.2f}".format(m[0]))
        print("Recall@5: {:.2f}".format(m[1]))
        print("Recall@10: {:.2f}".format(m[2]))
        print("Recall@top1: {:.2f}".format(m[3]))
        print("Recall@AP: {:.2f}".format(m[4]))
        # return m

        

In [4]:
m = MLPN_()
m.batchsize=128
m.test()

951
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:12<00:00, 3013.90it/s]


Recall@1: 91.75
Recall@5: 97.59
Recall@10: 98.44
Recall@top1: 98.54
Recall@AP: 93.05


(tensor(91.7475),
 tensor(97.5882),
 tensor(98.4361),
 tensor(98.5418),
 93.05280791725373)

In [11]:
m = MLPN_()
m.batchsize=128

from utils.loader import environments 

for style in environments:
    print(style)
    m.test(style=style, pth='net_170')

normal
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:13<00:00, 2883.25it/s]


Recall@1: 94.07
Recall@5: 98.34
Recall@10: 98.79
Recall@top1: 98.86
Recall@AP: 95.03
dark
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:12<00:00, 2924.97it/s]


Recall@1: 93.92
Recall@5: 98.32
Recall@10: 98.85
Recall@top1: 98.90
Recall@AP: 94.91
fog
load model: net_170.pth


...

Recall@1: 91.23
Recall@5: 97.36
Recall@10: 98.33
Recall@top1: 98.40
Recall@AP: 92.61
light
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:13<00:00, 2888.61it/s]


Recall@1: 90.40
Recall@5: 97.35
Recall@10: 98.34
Recall@top1: 98.44
Recall@AP: 91.96
wind
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:13<00:00, 2909.69it/s]

Recall@1: 89.58
Recall@5: 96.65
Recall@10: 97.87
Recall@top1: 98.00
Recall@AP: 91.17


trained on original dataset

In [10]:
for style in environments:
    print(style)
    m.test(style=style, pth='net_170_')

normal
load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:15<00:00, 2490.53it/s]


Recall@1: 93.71
Recall@5: 97.87
Recall@10: 98.54
Recall@top1: 98.60
Recall@AP: 94.69
dark
load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:15<00:00, 2453.34it/s]


Recall@1: 92.21
Recall@5: 97.69
Recall@10: 98.48
Recall@top1: 98.57
Recall@AP: 93.45
fog
load model: net_170_.pth


...

Recall@1: 37.43
Recall@5: 56.25
Recall@10: 64.51
Recall@top1: 65.67
Recall@AP: 41.99
light
load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:15<00:00, 2402.24it/s]


Recall@1: 85.42
Recall@5: 95.12
Recall@10: 97.03
Recall@top1: 97.24
Recall@AP: 87.59
wind
load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:15<00:00, 2452.96it/s]

Recall@1: 78.20
Recall@5: 91.55
Recall@10: 94.54
Recall@top1: 94.82
Recall@AP: 81.21


In [13]:
m.test(style='mixed', pth='net_170_')

load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:14<00:00, 2526.40it/s]


Recall@1: 63.14
Recall@5: 76.16
Recall@10: 80.78
Recall@top1: 81.38
Recall@AP: 66.21
